In [1]:
import os
import sys
import datetime as dt
import requests
import autofit.models as m
from config import configuration
import autofit.utils
import autofit.proxy as proxy

2019-07-22 23:37:12,065 - autofit.utils - INFO - importing /home/renzo/python/autofit/slots/__init__.pyc from autofit.slots
2019-07-22 23:37:12,070 - autofit.utils - INFO - importing /home/renzo/python/autofit/producers/__init__.pyc from autofit.producers
2019-07-22 23:37:12,074 - autofit - INFO - Network was successfully linked


In [2]:
# os.remove(os.path.join('db', 'mrprdev.db'))
m.create_all(configuration['development'])

In [ ]:
session = m.get_session()
for i in range(1):
    keys = '%s|EUR|euro|equity|index' % (i+1)
    slot = m.Slot(name='european_totem', keys=keys, date=dt.datetime.utcnow().date(), priority=1, state=m.SlotStatesEnum.pending, dbid='1')
    m.insert_slot(slot, None)


In [ ]:
proxy.schedule_producers()

In [ ]:
proxy.start_producers()

In [ ]:
p = session.query(m.Producer).all()

In [ ]:
p

In [ ]:
p[0].completed.set()
session.commit()

In [ ]:
slot1 = m.Slot(name='implied_fwd', keys=keys, date=dt.datetime.utcnow().date(), priority=1, state=m.SlotStatesEnum.pending, dbid='1')
slot2 = m.Slot(name='implied_vol', keys=keys, date=dt.datetime.utcnow().date(), priority=1, state=m.SlotStatesEnum.pending, dbid='1')

In [ ]:
m.insert_slot(slot1, p[0])
m.insert_slot(slot2, p[0])

In [ ]:
proxy.schedule_producers()

In [ ]:
proxy.start_producers()

In [ ]:
slot3 = m.Slot(name='product_rate', keys=keys, date=dt.datetime.utcnow().date(), priority=1, state=m.SlotStatesEnum.pending, dbid='4')
slot4 = m.Slot(name='dividends', keys=keys, date=dt.datetime.utcnow().date(), priority=1, state=m.SlotStatesEnum.pending, dbid='2')
slot5 = m.Slot(name='volatility', keys=keys, date=dt.datetime.utcnow().date(), priority=1, state=m.SlotStatesEnum.pending, dbid='3')

In [ ]:
# p = session.query(m.Producer).filter(m.Producer.running()).all()
p = session.query(m.Producer).filter(m.Producer.pending()).filter(m.Producer.delayed_by>0).all()

In [ ]:
print session.query(m.Producer).filter(m.Producer.pending()).filter(m.Producer.delayed_by>0)

In [ ]:
for x in p:
    print x.pname
    x.completed.set()
session.commit()

In [ ]:
m.insert_slot(slot3, p[0])

In [ ]:
m.insert_slot(slot4, p[0])

In [ ]:
m.insert_slot(slot5, p[1])

In [ ]:
proxy.schedule_producers()

In [ ]:
proxy.start_producers()

In [ ]:
s1 = session.query(m.Slot).all()[1]

In [ ]:
s1.listeners

In [ ]:
session = m.get_session()

In [ ]:
p = session.query(m.Producer).all()

In [ ]:
len(p)

In [ ]:
for x in session.query(m.Producer).subquery():
    x._asdict()

In [ ]:
p[0]._asdict()

In [3]:
session = m.get_session()

In [4]:
slot = session.query(m.Slot).filter_by(sid='7c4ca37b2525493e90eefef81e8aa0c2').one()

In [5]:
slot

<Slot(sid=7c4ca37b2525493e90eefef81e8aa0c2, name='varswapvolbasis', keys='1|EUR|euro|equity|index', date=2019-06-24, priority=1, state='current', dbid='1fa3ebdc90c2454d9af9fe37bcdb9e2b')>

In [7]:
l = m.lineage(slot)

In [9]:
l['slot']

{'keys': u'1|EUR|euro|equity|index',
 'name': u'varswapvolbasis',
 'pid': u'16e443c53d644de8876d7ac5698f2e2f',
 'pname': u'varswap_fitter',
 'sid': u'7c4ca37b2525493e90eefef81e8aa0c2'}

In [11]:
l['inputs'][1]

{'inputs': [{'inputs': [{'inputs': None,
     'slot': {'keys': u'1|EUR|euro|equity|index',
      'name': u'european_totem',
      'sid': u'b086eb7627f34d4e8f28a48aaf80e163'}}],
   'slot': {'keys': u'1|EUR|euro|equity|index',
    'name': u'implied_fwd',
    'pid': u'c5440afd04c041cdad067c943f7080af',
    'pname': u'black_scholes_calculator',
    'sid': u'641a376dc8854ec3a31289d3cbf42840'}}],
 'slot': {'keys': u'1|EUR|euro|equity|index',
  'name': u'dividends',
  'pid': u'8122c12e6fc042d3a06aa48d3a9b6075',
  'pname': u'synthetic_fwd_fitter',
  'sid': u'bcff3b4dd04e45d68fbe58e7e562f76a'}}